# Chargement du modèle

In [2]:
# Load libraries
import sklearn
import joblib

# Get scikit-learn version
scikit_version = sklearn.__version__

# Load the model
pipe = joblib.load("models/model_{version}.pkl".format(version=scikit_version))

# Save the model as pickle file for the web app
joblib.dump(pipe, "web/models/model_{version}.pkl".format(version=scikit_version))

# display the model
pipe

FileNotFoundError: [Errno 2] No such file or directory: 'models/model_1.1.1.pkl'

# Préparation des données transformées